### **Imports**

In [ ]:
from robolab_turtlebot import Turtlebot
from TurtleControllers import TurtleController
from TurtleVision import TurtleVision
import TurtleUtils

## **Initialization**

In [ ]:
turtle = Turtlebot(rgb = True, pc = True)
turtle_controller = TurtleController(turtle, rate = 40)
turtle_vision = TurtleVision(turtle)

#### **Find garage center**

In [ ]:
def reg_mean(regs):
    s, n = 0.0, len(regs) * 2
    for reg in regs:
        s += reg[0] + reg[1]
    return s / n

def reg_mid(reg):
    return (reg[0] + reg[1]) / 2

# Find Garage center
while True:
    regs = turtle_vision.get_regions(color = "purple")

    # Want to see two purple pillars
    while len(regs) < 2:
        turtle_controller.cmd_velocity(angular = 0.5)
        
    # Want to look at the center of the garage entrance
    m = reg_mean(regs)
    
    err = m - turtle_vision.img_width
    if abs(err) < 6:
        turtle_controller.cmd_velocity(linear = 0, angular = 0)
        break
    else:
        if err < 0.0:
            err = np.clip(err, -turtle_controller.max_yaw_rate, -turtle_controller.min_yaw_rate)
        else:
            err = np.clip(err, turtle_controller.min_yaw_rate, turtle_controller.max_yaw_rate)
        turtle_controller.cmd_velocity(linear = 0, angular = angular)
        
    turtle_controller.rate.sleep()

#### **Generate a garage model**

In [ ]:
turtle_controller.reset_odometry()
P1, P2, P_G, P_PG, P_GOAL = turtle_vision.garage_entry_waypoints()

fig, ax = plt.subplots(1)
TurtleUtils.visualize_garage(ax, P1, P2, P_G, P_PG, P_GOAL)

#### **Enter garage**

In [ ]:
for point in (P_PG, P_GOAL):
    turtle_controller.move_to(point, relative = False)